In [2]:
import sqlite3
import pandas as pd
import numpy as np

In [28]:
con = sqlite3.connect('items.db')
cur = con.cursor()


## Ozon

In [4]:
cur.execute('SELECT * FROM main.items_ozon')
res = cur.fetchall()
columns = ['itemid', 'externalid', 'link', 'img', 'name', 'brand', 'rating', 'countreviews']
df_ozon = pd.DataFrame(res, columns=columns)

## Lamoda

In [5]:
cur.execute('SELECT * FROM main.items_lamoda')
res = cur.fetchall()
columns = ['itemid', 'link', 'img', 'name', 'brand', 'rating', 'countreviews']
df_lamoda = pd.DataFrame(res, columns=columns)

In [6]:
def clean_name(x):
    res = ' '.join(sorted(x.lower().replace('кроссовки', '').replace('кеды', '').strip().split()))
    return res

In [7]:
df_ozon_ = df_ozon[['itemid', 'name', 'brand']]
df_ozon_['name'] = df_ozon_['name'].apply(clean_name)
df_ozon_ = df_ozon_.sort_values(by=['brand']).reset_index(drop=True)
df_ozon_['source'] = 'ozon'

/var/folders/cb/n7klvybs09g2ycl4j5z4b5800000gn/T/ipykernel_1651/3058028791.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ozon_['name'] = df_ozon_['name'].apply(clean_name)


In [8]:
df_lamoda_ = df_lamoda[['itemid', 'name', 'brand']]
df_lamoda_['name'] = df_lamoda_['name'].apply(clean_name)
df_lamoda_ = df_lamoda_.sort_values(by=['brand']).reset_index(drop=True)
df_lamoda_['source'] = 'lamoda'

/var/folders/cb/n7klvybs09g2ycl4j5z4b5800000gn/T/ipykernel_1651/688256556.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lamoda_['name'] = df_lamoda_['name'].apply(clean_name)


In [9]:
df = pd.merge(df_lamoda_, df_ozon_, how='outer', on='name')

In [10]:
df = df.dropna(subset=['itemid_x', 'itemid_y'])

In [11]:
res_df = df[~df['name'].isin(['два мяча', 'patrol', 'sigma', 'timejump', 's.oliver'])][['itemid_x', 'itemid_y']]

In [12]:
res_df = res_df.rename(columns={'itemid_x':'id_lamoda', 'itemid_y':'id_ozon'})

In [21]:
con.execute('''CREATE TABLE items_matching
            (itemid_lamoda integer, itemid_ozon integer)
                        ''')

In [22]:
cur.executemany('INSERT INTO items_matching (itemid_lamoda, itemid_ozon) VALUES(?, ?)', res_df.values.tolist())

In [23]:
cur.execute('select * from items_matching')

In [30]:
con.commit()

In [31]:
con.close()